In [2]:
import matplotlib.pyplot as plt
import pandas as pd

In [33]:
class AmortizationSimulator:
    def __init__(self, home_price, loan_amount, loan_interest_rate, 
                 loan_length_in_years, taxes_and_insurance):
        self.home_price = home_price
        self.loan_amount = loan_amount
        self.loan_interest_rate = loan_interest_rate
        self.loan_length_in_years = loan_length_in_years
        self.taxes_and_insurance = taxes_and_insurance
        self.monthly_payment = self.monthly_mortgage()
        self.down_payment = home_price - loan_amount
        
    def monthly_mortgage(self):
        """ 
        monthly mortgage calculator: 
        M= P[r(1+r)^n/((1+r)^n)-1)]
        """
        monthly_interest_rate = self.loan_interest_rate/12
        num_payments = self.loan_length_in_years * 12
        interest_scale =(1+monthly_interest_rate) ** num_payments 
        numerator = monthly_interest_rate * interest_scale 
        denominator = interest_scale-1 
        monthly = self.loan_amount * (numerator/denominator)
        return monthly
    
    def remaining_principal(self, principal, num_payments, extra_on_principal = 0):
        if num_payments > self.loan_length_in_years * 12:
            raise Exception('num_payments must be less than or equal to ' + str(self.loan_length_in_years * 12))
        if num_payments == 0:
            return principal
        interest_payment = principal * (self.loan_interest_rate/12)
        principal_payment = self.monthly_payment - interest_payment
        new_principal = principal - principal_payment - extra_on_principal
        num_payments = num_payments - 1
        return self.remaining_principal(new_principal, num_payments, extra_on_principal)
    
    def home_value(self, growth_rate, num_payments):
        new_value = self.home_price * (growth_rate ** (num_payments/12))
        return new_value 
    
    def summary(self, growth_rate, num_payments, rental_price = 0, extra_on_principal = 0):
        remaining = self.remaining_principal(self.loan_amount, num_payments, extra_on_principal)
        gained_equity = 1 - remaining/self.loan_amount
        estimated_new_home_price = self.home_value(growth_rate, num_payments)
        estimated_home_profit = estimated_new_home_price - self.home_price
        rental_gains = (rental_price - (self.monthly_payment + self.taxes_and_insurance) - extra_on_principal) * num_payments

        print('Down payment: ' + str(round(self.down_payment,2)))
        print('Monthly mortgage: ' + str(round(self.monthly_payment,2)))
        print('Monthly mortgage and fees: ' + str(round(self.monthly_payment + self.taxes_and_insurance,2)))
        print()
        print('After ' + str(num_payments) + ' payments...')
        print()
        print('Paid: ' + str( round(num_payments * (self.monthly_payment + self.taxes_and_insurance),2)))
        print('Paid towards principal: ' + str(round(self.loan_amount - remaining,2)))
        print('Gained equity: ' + str(round((gained_equity * 100),2)) + '%')
        print()
        print('Assuming ' + str(round((growth_rate-1)*100,2)) + '% annual home value growth...')
        print('Home price : ' + str(round(estimated_new_home_price,2)))
        print('Gained equity : ' + str(round(estimated_home_profit,2)))
        print()
        print('Total equity from principal and growth: ' + str( round((self.home_price * gained_equity) + estimated_home_profit,2)) )
        print()
        print('Assuming rental price of ' + str(rental_price) + '...')
        print('Gained rental profit ' + str(round(rental_gains,2)))
        print()
        print('Total gained ' + str(round(rental_gains +  (self.home_price * gained_equity) + estimated_home_profit,2)))       
        print('ROI ' + str(round((rental_gains +  (self.home_price * gained_equity) + estimated_home_profit) / (self.home_price-self.loan_amount),2)*100))
              
              
              
              
              
              

In [24]:
# statics
home_price = 680000
closing_costs = 2701.43
loan_amount = 544000
loan_interest_rate = 0.0275
loan_length_in_years = 30
taxes_and_insurance = 761.16

# time
num_payments = 120

# assumptions
growth_rate = 1.03
rental_price = 3295
extra_on_principal = 0  # amount of extra (from rent) to contribute on principal? 

home_amortization = AmortizationSimulator(home_price = home_price, 
                      loan_amount = loan_amount, 
                      loan_interest_rate = loan_interest_rate,
                      loan_length_in_years = loan_length_in_years,
                      taxes_and_insurance = taxes_and_insurance
                      )

home_amortization.home_value(num_payments = num_payments, growth_rate = growth_rate)

home_amortization.summary(num_payments = num_payments, 
                          growth_rate = growth_rate, 
                          rental_price = rental_price, 
                          extra_on_principal = extra_on_principal)

Down payment: 136000
Monthly mortgage: 2220.83
Monthly mortgage and fees: 2981.99

After 120 payments...

Paid: 357839.04
Paid towards principal: 134378.08
Gained equity: 24.7%

Assuming 3.0% annual home value growth...
Home price : 913863.14
Gained equity : 233863.14

Total equity from principal and growth: 401835.74

Assuming rental price of 3295...
Gained rental profit 37560.96

Total gained 439396.7


In [18]:
# University Park 90007, YTY growth. 
YTY = [423, 458, 511, 580, 672, 722, 750, 795]
YTY_change = []
for k in range(0,len(YTY)-1):
    YTY_change.append(round(YTY[k+1]/YTY[k],2))
    
import numpy as np
print(YTY_change)
print(np.mean(YTY_change))

[1.08, 1.12, 1.14, 1.16, 1.07, 1.04, 1.06]
1.0957142857142856


In [22]:
gained_equity = []
downpayment_pct = 28.9
for payment_num in range(1,361):
    remaining = home_amortization.remaining_principal(principal = 544000, num_payments = payment_num)
    total_equity = round(((1 - remaining/544000)*80) + downpayment_pct,2)
    gained_equity.append(total_equity)
pd.DataFrame(gained_equity[11:]).to_csv('test.csv')


In [34]:
salary = 95000
bonus = 1.15
individual = .11
employer = .04

indiv_contribution = (salary * individual) 
company_contribution = (salary * bonus * employer)
total = indiv_contribution + company_contribution
print (indiv_contribution, company_contribution, total)

10450.0 4369.999999999999 14820.0


In [30]:
(indiv_contribution + company_contribution) / indiv_contribution

1.4181818181818182